In [5]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import concurrent.futures
import time

### Functions that Will Extract Location and Address Data

In [2]:
##this function extracts the location (City State and Zipcode) for each school.

def extractLocation(soup):
    address_element = soup.select_one('address') 
        #print(address_element)
        # Exclude the text within the span within the address element
    if address_element:
        city_state = address_element.find('span')
    
    else:
        #print(f"Failed to retrieve the webpage. Status code: {response.status_code}")
        return None  # Return None or some default value if the request fails

    return city_state.text

In [3]:
#this function extracts the address for each school.

def extractAddress(soup):
        # Find the address element
    address_element = soup.select_one('address') 
        #print(address_element)
        # Exclude the text within the span within the address element

    city_state = address_element.find('span')
    city_state.decompose()

    address_text = address_element.get_text(strip=True)
    if address_text != '':
        return address_text
    else:
        return None

In [4]:
#this function cleans up the location column by splitting it into city, state and zipcode columns.
def splitLocation(column):
    
    pattern = r'(?P<city>[^,]+).\s?(?P<state>\w\w)\s?(?P<zipcode>\d\d\d\d\d)?'
    #column = pd.Series(column)
    _city = column.str.extract(pattern)['city']
    _state = column.str.extract(pattern)['state']
    _zip = column.str.extract(pattern)['zipcode']
    
    return (_city, _state, _zip)

### Functions that extract the Score and Game info

In [5]:
#this funciton cleans up the gameinfo column by splitting it into venue, team2 and gametype (H or A) columns.
def gameInfo(column):
    pattern = r'(?P<venue>[vs@]*)(?P<name>[a-zA-Z]+(\.|-)?\s?[a-zA-Z]*\'?\w*)(?P<type>[*]*)'
    # Apply str.extract to create new columns based on the pattern
    _venue = column.str.extract(pattern)['venue']
    _team2 = column.str.extract(pattern)['name']
    _gametype = column.str.extract(pattern)['type']
    
    
    return (_venue, _team2, _gametype)

In [6]:
#this function cleans up the score column by splitting it into team, team1_score and team2_score columns.
def scoreInfo(column):
    # Define a regular expression pattern to extract W or L, and the scores
    pattern = r'([WL])(\d+)-(\d+)|([WL])\(FF\)'

    # Extract the components into separate columns
    extracted = column.str.extract(pattern)

    # Assign extracted values to new columns
    team = extracted[0].fillna(extracted[3])
    team1_score = extracted[1]#.fillna(pd.NA).astype('Int64')
    team2_score = extracted[2]#.fillna(pd.NA).astype('Int64')

    return team, team1_score, team2_score

### Functions that format the game and venue type correctly

In [7]:
#each game entry will have  *, **, or *** next to it which correponds to the game type. This function cleans up the game type column.
def cleanGameType(string):
    #string = str(len(string))
    if len(string) == 3:
        string = 'Tournament'
    elif len(string) == 2:
        string = 'Playoff'
    elif len(string) == 1:
        string = 'District'
    else:
        string = 'Regular Season'
    return string

In [8]:
def cleanVenueType(string):
    if string == '@':
        return 'A'
    return 'H'

### Making a Dict with each school's name and URL

In [9]:
#this function extracts the number of pages on Maxpreps for each state.
def get_num_pages(state):
    current_page = 1
    while True:
        url = f'https://www.maxpreps.com/{state}/basketball/21-22/rankings/{current_page}/'
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        next_page_button = soup.find_all('a', class_ = 'btn btn-default')
        button = [butt.text for butt in next_page_button]
        num = str(current_page + 1)
        if num not in button:
            return int(num)- 1
        current_page += 1

In [10]:
#this function will extract 
def extract_state_ranking_page(state, current_page):
    url = f'https://www.maxpreps.com/{state}/basketball/21-22/rankings/{current_page}/'
    response = requests.get(url)
    soup = BeautifulSoup(response.text , 'html.parser')
    school_elements = soup.find_all('th', class_='school')
    school_elements.pop(0)
    school_links = [school.find('a').get('href') for school in school_elements]
    return dict(zip((school.text for school in school_elements), school_links))

In [11]:
#this function will create a dictionary of schools and their respective links given a state.
def create_school_dict(state):
    school_dict = {}
    num_pages = get_num_pages(state)
    with concurrent.futures.ThreadPoolExecutor() as executor:
        future_to_page = {executor.submit(extract_state_ranking_page, state, page): page for page in range(1, num_pages + 1)}
        for future in concurrent.futures.as_completed(future_to_page):
            try:
                page = future_to_page[future]
                data = future.result()
                school_dict.update(data)
            except Exception as e:
                print(f"Error fetching page {page}: {e}")
    return school_dict

### Mascot Extraction

In [12]:
#The purpose of extracting the mascot for each school is to give each school a unique indentifer that will later be used get the location and address of team 2.
def extractMascots(soup):
    src_links = []
    #soup = BeautifulSoup(html_content, 'html.parser')
    div_elements = soup.find_all('div', class_='sc-3a367303-0 guEKkV')
    
    for div in div_elements:
        img_element = div.find('img', class_='sc-e055731c-0 TgzVI photo-or-initial')
        if img_element:
            src_links.append(img_element.get('src'))
        else:
            src_links.append(None)
    
    return src_links

In [13]:
def formatMascot(string):
    if string is not None:
        output_string = string.replace('width=32&height=32', 'width=64&height=64')
        return output_string
    return None

### Sports Offered Extraction

In [14]:
#this function extracts the sports offered at a given school for both boys and girls. It takes in the school URL as input
def extractSports(url):
    url = f'http://www.maxpreps.com{url}'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'lxml')

    elements = soup.find_all('div', class_ = 'sports-list-child')

    boys = []
    girls = []
    #check if school offers no sports
    if len(elements) == 0:
        return None

    for element in elements:
        gender = element.find('h2', class_ = 'sc-23c90da8-0 KWggY')
        sports = element.find_all('span', class_ = 'sport-name')
        if gender.text == 'Boys':
            for sport in sports:
                boys.append(sport.text)
        else:
            for sport in sports:
                girls.append(sport.text)
                
    return ','.join(boys), ','.join(girls)

### Merging DF

In [15]:
#This function will create a df that has the location and address of team 2. Merging was perfomed on the mascot column.
def mergeDF(df):
    df['Team 1 Mascot'] = df['Team 1 Mascot'].astype(str)
    df['Team 2 Mascot'] = df['Team 2 Mascot'].astype(str)
    df['Team 2 Mascot'] = df['Team 2 Mascot'].apply(formatMascot)
    X = df.loc[:, ['Team 1 Address', 'Team 1 City', 'Team 1 State', 'Team 1 Zipcode', 'Team 1 Mascot']]
    X = X.rename(columns={'Team 1 Address': 'Team 2 Address',
                                   'Team 1 City': 'Team 2 City',
                                   'Team 1 State': 'Team 2 State',
                                   'Team 1 Zipcode': 'Team 2 Zipcode',
                                     'Team 1 Mascot': 'Team 2 Mascot'})
    X = X.drop_duplicates()
    X = X[X['Team 2 Mascot'] != 'None']
    merged_df = pd.merge(df, X, how = 'left', left_on = 'Team 2 Mascot', right_on = 'Team 2 Mascot')
    
    return merged_df

### Put it all together

In [16]:
import concurrent.futures
from tenacity import retry, stop_after_attempt, wait_fixed
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor

In [17]:
from tenacity import retry, stop_after_attempt, wait_fixed

In [18]:
#this function utilizes all previous functions. it will read in the school name and url, scrape the data, and clean everything up to a useable format.

@retry(stop=stop_after_attempt(5), wait=wait_fixed(3))
def scrape_school_data(school, url):
    try:
        school_url = f'http://www.maxpreps.com{url}'
        #print(school_url)
        response = requests.get(school_url).text
        soup = BeautifulSoup(response, 'lxml')
        table = soup.find('table')
        data = pd.read_html(str(table))[0]
        data = data.drop('Game Info', axis=1)
        data['Team 1'] = school
        data['Team 1 Location'] = extractLocation(soup)
        data['Team 1 Address'] = extractAddress(soup)
        data['Team 1 City'] = splitLocation(data['Team 1 Location'])[0]
        data['Team 1 State'] = splitLocation(data['Team 1 Location'])[1]
        data['Team 1 Zipcode'] = splitLocation(data['Team 1 Location'])[2]
        data['Venue'] = gameInfo(data['Opponent'])[0].apply(cleanVenueType)
        data['Team 2'] = gameInfo(data['Opponent'])[1]
        data['Game Type'] = gameInfo(data['Opponent'])[2].apply(cleanGameType)
        data['Outcome'] = scoreInfo(data['Result'])[0]
        data['Team 1 Score'] = scoreInfo(data['Result'])[1]
        data['Team 2 Score'] = scoreInfo(data['Result'])[2]
        team1_mascot = soup.find('img', class_ = 'sc-e055731c-0 ddWTnk photo-or-initial')
        if team1_mascot:  
            data['Team 1 Mascot'] = team1_mascot.get('src')
        else:
            data['Team 1 Mascot'] = None
        team_2_mascots = extractMascots(soup)
        data['Team 2 Mascot'] = team_2_mascots
        sports_url = url.rsplit('/', 4)[0] + '/'
        #print(sports_url)
        sports_offered = extractSports(sports_url)
        data['Boys Sports'] = sports_offered[0]
        data['Girls Sports'] = sports_offered[1]

        return data

    except Exception as e:
        print(f"Error processing {school}: {e}")
        return None

In [19]:
#this function is given a state and a dictionary of schools and their urls. It will scrape all the schools in the dictionary and return a dataframe with all the data.
def scrape_all_schools_async(school_dict, state):

    grand_df = pd.DataFrame()

    with ThreadPoolExecutor() as executor:
        futures = [executor.submit(scrape_school_data, school, url) for school, url in school_dict.items()]

        with tqdm(total=len(futures), desc=f"Scraping Schools for {state}", unit=" schools") as pbar:
            for future in concurrent.futures.as_completed(futures):
                data = future.result()
                if data is not None:
                    grand_df = pd.concat([grand_df, data], ignore_index=True)
                pbar.update(1)

    return grand_df

### Running It

In [28]:
cols = ['Date', 'Team 1', 'Team 2', 'Venue', 'Game Type', 'Team 1 Score', 
       'Team 2 Score', 'Outcome', 'Team 1 Address', 'Team 1 City',
       'Team 1 State', 'Team 1 Zipcode','Team 2 Address','Team 2 City',
        'Team 2 State', 'Team 2 Zipcode', 'Boys Sports', 'Girls Sports', ]
merged = merged[cols]

In [29]:
null_values = merged.isnull()

# Sum the null values for each column
null_counts = null_values.sum()

In [53]:
states = [
    "ak", "al", "ar", "az", "ca", "co", "ct", "dc", "de", "fl", "ga",
    "hi", "ia", "id", "il", "in", "ks", "ky", "la", "ma", "md",
    "me", "mi", "mn", "mo", "ms", "mt", "nc", "nd", "ne", "nh",
    "nj", "nm", "nv", "ny", "oh", "ok", "or", "pa", "ri", "sc",
    "sd", "tn", "tx", "ut", "va", "vt", "wa", "wi", "wv", "wy"
]

#this loop:
#(1) iterates through each state
#(2) in each iteration a school dictionary is created with school as key and url as value
#(3) the data is scraped for each school in the dictionary
#(4) Team 2 Location and Address is addeed using the mergeDF function
#(5) columns are reordered
#(6) the data for each state is saved in its own csv file
for state in states:
    school_dict = create_school_dict(state)
    school_dict = {k: school_dict[k] for k in sorted(school_dict)}
    #print(school_dict)
    data = scrape_all_schools_async(school_dict, state)
    merged = mergeDF(data)
    cols = ['Date', 'Team 1', 'Team 2', 'Venue', 'Game Type', 'Team 1 Score', 
       'Team 2 Score', 'Outcome', 'Team 1 Address', 'Team 1 City',
       'Team 1 State', 'Team 1 Zipcode','Team 2 Address','Team 2 City',
        'Team 2 State', 'Team 2 Zipcode', 'Boys Sports', 'Girls Sports',
        'Team 1 Mascot', 'Team 2 Mascot']
    merged = merged[cols]
    merged.to_csv(f'{state}.csv', index=False)

Scraping Schools for az:  59%|█████▉    | 193/328 [00:29<00:19,  6.79 schools/s]

Error processing PHH Prep Open (Phoenix): No tables found
Error processing PHH Prep National (Phoenix): No tables found


Scraping Schools for ca:  50%|████▌    | 718/1428 [01:49<02:09,  5.49 schools/s]

Error processing Liberty Charter (Alpine): HTTPConnectionPool(host='www.maxpreps.com', port=80): Max retries exceeded with url: /ca/alpine/liberty-charter-lions/ (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fc2e0ca9a00>: Failed to establish a new connection: [Errno 60] Operation timed out'))Error processing Liberty (Madera): HTTPSConnectionPool(host='www.maxpreps.com', port=443): Max retries exceeded with url: /ca/madera/liberty-hawks/ (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fc2dfc20f70>: Failed to establish a new connection: [Errno 60] Operation timed out'))



Scraping Schools for ca:  64%|█████▋   | 907/1428 [02:20<01:09,  7.51 schools/s]

...

Scraping Schools for tx:  64%|█████   | 1080/1688 [02:49<01:47,  5.65 schools/s]

Error processing Montgomery: HTTPConnectionPool(host='www.maxpreps.com', port=80): Max retries exceeded with url: /tx/montgomery/montgomery-bears/ (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fc2e6bffd60>: Failed to establish a new connection: [Errno 60] Operation timed out'))


Scraping Schools for tx:  68%|█████▍  | 1140/1688 [03:00<01:55,  4.73 schools/s]

Error processing Parkview Christian (Waco): No tables found


Scraping Schools for wy: 100%|████████████| 67/67 [00:09<00:00,  6.78 schools/s]
